In [1]:
!pip install jupyter-plotly-dash

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Setup the Jupyter version of Dash
from jupyter_plotly_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = 'aacuser'
password = 'cs340password'
host = 'nv-desktop-services.apporto.com'
port = 30113
database_name = 'AAC'
collection_name = 'animals'

# Connect to database via CRUD Module
shelter = AnimalShelter(database_name, collection_name, username, password, host, port)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'grazioso.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    href='https://www.snhu.edu'
    ),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Jarrod Thomisee'))),
    html.Center(html.H2("Animal Shelter Data Management Dashboard", style={'color': 'blue'})),
    html.Div([
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'wilderness_rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'  # Default value
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable='single',
        row_selectable='single',
        selected_columns=[],
        selected_rows=[0],
        page_action='native',
        page_current=0,
        page_size=10,
    ),
    html.Br(),
     html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),           
     Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'water_rescue':
        data = shelter.water_rescue_query()
    elif filter_type == 'wilderness_rescue':
        data = shelter.mountain_rescue_query()
    elif filter_type == 'tracking':
        data = shelter.disaster_tracking_query()
    else:
        data = shelter.read({})  # Default case for 'Reset'

    filtered_df = pd.DataFrame.from_records(data)
    filtered_df.drop(columns=['_id'], inplace=True)
    data = filtered_df.to_dict('records')
    
    # Check if data is not empty and then select the first row
    if data:
        return data, [0]
    else:
        return [], []

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return

    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Animals')
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app
